In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,NaN,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,NaN,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,NaN,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1990-06-07,84.0,0.0,0.0,0.0,0.0,4.0,1.0,50.

In [3]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

In [9]:
# player_stats['Match_ID'].unique()

In [10]:
match = player_stats[player_stats['Match_ID'] == "2021F4_Melbourne_WesternBulldogs"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
9437,2021F4_Melbourne_WesternBulldogs,Melbourne,Alex Neal-Bullen,2021F4,Alex_Neal-Bullen,MEDIUM_FORWARD,CD_I296420,26,183,80,30,RIGHT,SA,2014.0,2015.0,Plympton (SA)/Glenelg (SANFL),40.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1996-01-09,88.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,34.8,8.0,0.0,0.0,0.0,11.0,75.0,24.0,89.0,18.0,7.0,4.0,0.0,0.0,33.3,1.0,1.0,8.0,13.0,0.0,0.0,0.0,0.0,2.0,1.0,5.0,63.6,0.0,0.0,11.0,0.8,5.0,1.0,0.0,292.0,0.0,25.0,14.8,1.0,0.0,10.0,2.0,3.0,0.0,1.0,2.0,0.0,1.0,23.0,4.0,15.0,NaN,NaN,Forward Pocket Left,FINAL_TEAM,Forward-Pocket,Forward,2021.0,NaN,2021,7.141076,-0.044595,0.062357,0.511760,0.740851,-0.229090,4.557391,5.777632,-1.220240,0.246734,0.141593,0.084702,-2.973776,Alex Neal-Bullen_2021,7.0,-0.141076,Melbourne,Western Bulldogs,Western Bulldogs
9438,2021F4_Melbourne_WesternBulldogs,Melbourne,Angus Brayshaw,2021F4,Angus_Brayshaw,MIDFIELDER,CD_I298279,26,187,91,10,RIGHT,VIC,2014.0,2015.0,Hampton Rovers (Vic)/Sandringham U18,3.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1996-01-09,85.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,39.3,11.0,0.0,0.0,0.0,5.0,60.0,25.0,95.0,15.0,9.0,1.0,0.0,1.0,50.0,1.0,1.0,6.0,10.0,0.0,0.0,0.0,0.0,7.0,0.0,8.0,60.0,0.0,0.0,15.0,1.5,5.0,1.0,0.0,413.0,6.0,10.0,13.2,1.0,0.0,5.0,1.0,2.0,3.0,3.0,2.0,0.0,3.0,28.0,8.0,17.0,Angus_Brayshaw,1.0,Wing Left,FINAL_TEAM,Wing,Midfield,2021.0,NaN,2021,5.494351,0.130623,0.037481,1.155812,1.115936,0.039876,5.524129,5.145365,0.378764,0.120117,0.074379,0.732630,5.401938,Angus Brayshaw_2021,6.0,0.505649,Melbourne,Western Bulldogs,Western Bulldogs
9439,2021F4_Melbourne_WesternBulldogs,Melbourne,Bayley Fritsch,2021F4,Bayley_Fritsch,MEDIUM_FORWARD,CD_I1001438,25,188,84,31,LEFT,VIC,2017.0,2018.0,Coldstream (Vic)/Casey (VFL),31.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1996-12-06,79.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,35.7,5.0,3.0,0.0,0.0,1.0,61.5,13.0,87.0,8.0,7.0,3.0,1.0,1.0,75.0,1.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,70.0,0.0,0.0,10.0,3.3,5.0,4.0,3.0,256.0,0.0,10.0,16.4,0.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,14.0,3.0,9.0,Bayley_Fritsch,8.0,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,NaN,2021,29.425070,-0.155321,0.031260,3.172690,3.416951,-0.244260,9.095646,11.885356,-2.789709,1.066444,0.000000,3.072950,17.675953,Bayley Fritsch_2021,38.0,8.574930,Melbourne,Western Bulldogs,Western Bulldogs
9440,20

In [11]:
match_for = match.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_for.columns = [x+"_F" for x in list(match_for)]
match_against = match.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_against.columns = [x+"_A" for x in list(match_against)]
match_summary = pd.concat([match_for, match_against], axis=1)
for col in [x.replace("_F", "") for x in list(match_for)]:
        match_summary[col+"_diff"] = match_summary[col+"_F"] - match_summary[col+"_A"]
match_summary

,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_diff,xScore_diff,vaep_value_diff,vaep_value_received_diff,offensive_value_diff,defensive_value_diff,exp_vaep_value_diff,exp_vaep_value_received_diff,exp_offensive_value_diff,exp_defensive_value_diff
Melbourne,137.0,111.882439,23.793333,17.330982,22.542587,1.250746,130.231822,115.540981,125.028147,5.203677,64.0,56.065672,12.999723,10.435025,11.722368,1.277355,79.263204,79.806062,70.935042,8.328161,73.0,55.816767,10.79361,6.895957,10.82022,-0.02661,50.968618,35.734918,54.093104,-3.124485
Western Bulldogs,64.0,56.065672,12.999723,10.435025,11.722368,1.277355,79.263204,79.806062,70.935042,8.328161,137.0,111.882439,23.793333,17.330982,22.542587,1.250746,130.231822,115.540981,125.028147,5.203677,-73.0,-55.816767,-10.79361,-6.895957,-10.82022,0.02661,-50.968618,-35.734918,-54.093104,3.124485


In [15]:
home_team = match[match['Team'] == match['Home_Team']]
away_team = match[match['Team'] != match['Home_Team']]

home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
home_team['player_value'] = home_team['exp_vaep_value_%'] * (home_team['xScore'].sum())

away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
away_team['player_value'] = away_team['exp_vaep_value_%'] * (away_team['xScore'].sum())
match_player_values = pd.concat([home_team, away_team])
# match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'exp_vaep_value_%', 'player_value']].sort_values(by='player_value', ascending=False)

In [16]:
def create_player_value(match_player_stats):
    
    home_team = match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']]
    home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
    home_team['player_value'] = home_team['exp_vaep_value_%'] * home_team['xScore'].sum()

    away_team = match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']]
    away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
    away_team['player_value'] = away_team['exp_vaep_value_%'] * away_team['xScore'].sum()
    match_player_values = pd.concat([home_team, away_team])
    match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'player_value']].sort_values(by='player_value', ascending=False).head(10)

    return match_player_values

In [17]:
match_player_values_list = []
for match_id in list(player_stats['Match_ID'].unique()):
    
    match_player_stats = player_stats[player_stats['Match_ID'] == match_id]
    match_player_values = create_player_value(match_player_stats)
    match_player_values_list.append(match_player_values)
    
player_values = pd.concat(match_player_values_list)


In [18]:
player_values

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,exp_vaep_value_%,player_value
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,0.033436,2.331407
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney,0.013115,0.914469
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0.0,0.00